In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2021"

In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [4]:
tables = soup.find_all('table',class_='wikitable sortable')

In [5]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [6]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [7]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref,Ref.
0,JANUARY,1,Shadow in the Cloud,Vertical Entertainment,Roseanne Liang (director/screenplay); Max Land...,[2],NaN
1,JANUARY,13,The White Tiger,Netflix,Ramin Bahrani (director/screenplay); Adarsh Go...,NaN,NaN
2,JANUARY,14,Locked Down,HBO Max / Warner Bros. Pictures,Doug Liman (director); Steven Knight (screenpl...,[3],NaN
3,JANUARY,15,The Dig,Netflix / Clerkenwell Films,Simon Stone (director); Moira Buffini (screenp...,[4],NaN
4,JANUARY,15,Outside the Wire,Netflix,"Mikael Håfström (director); Rob Yescombe, Rowa...",[5],NaN
...,...,...,...,...,...,...,...
239,DECEMBER,17,Spider-Man: No Way Home,Columbia Pictures / Marvel Studios,"Jon Watts (director); Chris McKenna, Erik Somm...",NaN,[172]
240,DECEMBER,22,Untitled fourth Matrix film,HBO Max / Warner Bros. Pictures / Village Road...,Lana Wachowski (director/screenplay); Aleksand...,NaN,[173]
241,DECEMBER,22,Sing 2,Universal Pictures / Illumination,Garth Jennings (director/screenplay); Matthew ...,NaN,[174]
242,DECEMBER,22,The King's Man,20th Century Studios / Marv Films,Matthew Vaughn (director/screenplay); Karl Gaj...,NaN,[111]


In [8]:
df_2021 = df[['Title','Cast and crew']]

In [9]:
df_2021

,Title,Cast and crew
0,Shadow in the Cloud,Roseanne Liang (director/screenplay); Max Land...
1,The White Tiger,Ramin Bahrani (director/screenplay); Adarsh Go...
2,Locked Down,Doug Liman (director); Steven Knight (screenpl...
3,The Dig,Simon Stone (director); Moira Buffini (screenp...
4,Outside the Wire,"Mikael Håfström (director); Rob Yescombe, Rowa..."
...,...,...
239,Spider-Man: No Way Home,"Jon Watts (director); Chris McKenna, Erik Somm..."
240,Untitled fourth Matrix film,Lana Wachowski (director/screenplay); Aleksand...
241,Sing 2,Garth Jennings (director/screenplay); Matthew ...
242,The King's Man,Matthew Vaughn (director/screenplay); Karl Gaj...


In [10]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '1706f19c7062d21150c1c7620a5dce33'

In [11]:
from tmdbv3api import Movie
tmdb_movie = Movie() 
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/550?api_key=1706f19c7062d21150c1c7620a5dce33'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

In [12]:
df_2021['genres'] = df_2021['Title'].map(lambda x: get_genre(str(x)))

<ipython-input-12-252c1cda1e49>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['genres'] = df_2021['Title'].map(lambda x: get_genre(str(x)))


In [13]:
df_2021

,Title,Cast and crew,genres
0,Shadow in the Cloud,Roseanne Liang (director/screenplay); Max Land...,Drama
1,The White Tiger,Ramin Bahrani (director/screenplay); Adarsh Go...,Drama
2,Locked Down,Doug Liman (director); Steven Knight (screenpl...,Drama
3,The Dig,Simon Stone (director); Moira Buffini (screenp...,Drama
4,Outside the Wire,"Mikael Håfström (director); Rob Yescombe, Rowa...",Drama
...,...,...,...
239,Spider-Man: No Way Home,"Jon Watts (director); Chris McKenna, Erik Somm...",Drama
240,Untitled fourth Matrix film,Lana Wachowski (director/screenplay); Aleksand...,NaN
241,Sing 2,Garth Jennings (director/screenplay); Matthew ...,Drama
242,The King's Man,Matthew Vaughn (director/screenplay); Karl Gaj...,Drama


In [14]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [15]:
df_2021['director_name'] = df_2021['Cast and crew'].map(lambda x: get_director(str(x)))

<ipython-input-15-059d9cfe3ca6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['director_name'] = df_2021['Cast and crew'].map(lambda x: get_director(str(x)))


In [16]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [17]:
df_2021['actor_1_name'] = df_2021['Cast and crew'].map(lambda x: get_actor1(str(x)))

<ipython-input-17-fbf9a4980306>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['actor_1_name'] = df_2021['Cast and crew'].map(lambda x: get_actor1(str(x)))


In [18]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [19]:
df_2021['actor_2_name'] = df_2021['Cast and crew'].map(lambda x: get_actor2(str(x)))

<ipython-input-19-597eaf6de001>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['actor_2_name'] = df_2021['Cast and crew'].map(lambda x: get_actor2(str(x)))


In [20]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [21]:
df_2021['actor_3_name'] = df_2021['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [22]:
df_2021

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,Shadow in the Cloud,Roseanne Liang (director/screenplay); Max Land...,Drama,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale
1,The White Tiger,Ramin Bahrani (director/screenplay); Adarsh Go...,Drama,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas
2,Locked Down,Doug Liman (director); Steven Knight (screenpl...,Drama,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant
3,The Dig,Simon Stone (director); Moira Buffini (screenp...,Drama,Simon Stone,Carey Mulligan,Ralph Fiennes,Lily James
4,Outside the Wire,"Mikael Håfström (director); Rob Yescombe, Rowa...",Drama,Mikael Håfström,Anthony Mackie,Damson Idris,Emily Beecham
...,...,...,...,...,...,...,...
239,Spider-Man: No Way Home,"Jon Watts (director); Chris McKenna, Erik Somm...",Drama,Jon Watts,Tom Holland,Zendaya,Jacob Batalon
240,Untitled fourth Matrix film,Lana Wachowski (director/screenplay); Aleksand...,NaN,Lana Wachowski,Keanu Reeves,Carrie-Anne Moss,Yahya Abdul-Mateen II
241,Sing 2,Garth Jennings (director/screenplay); Matthew ...,Drama,Garth Jennings,Matthew McConaughey,Reese Witherspoon,Scarlett Johansson
242,The King's Man,Matthew Vaughn (director/screenplay); Karl Gaj...,Drama,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans


In [23]:
df_2021 = df_2021.rename(columns={'Title':'movie_title'})

In [24]:
new_df21 = df_2021.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [25]:
new_df21

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale,Drama,Shadow in the Cloud
1,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas,Drama,The White Tiger
2,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant,Drama,Locked Down
3,Simon Stone,Carey Mulligan,Ralph Fiennes,Lily James,Drama,The Dig
4,Mikael Håfström,Anthony Mackie,Damson Idris,Emily Beecham,Drama,Outside the Wire
...,...,...,...,...,...,...
239,Jon Watts,Tom Holland,Zendaya,Jacob Batalon,Drama,Spider-Man: No Way Home
240,Lana Wachowski,Keanu Reeves,Carrie-Anne Moss,Yahya Abdul-Mateen II,NaN,Untitled fourth Matrix film
241,Garth Jennings,Matthew McConaughey,Reese Witherspoon,Scarlett Johansson,Drama,Sing 2
242,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,Drama,The King's Man


In [26]:
new_df21['comb'] = new_df21['actor_1_name'] + ' ' + new_df21['actor_2_name'] + ' '+ new_df21['actor_3_name'] + ' '+ new_df21['director_name'] +' ' + new_df21['genres']

In [27]:
new_df21

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale,Drama,Shadow in the Cloud,Chloë Grace Moretz Taylor John Smith Beulah Ko...
1,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas,Drama,The White Tiger,Adarsh Gourav Rajkummar Rao Priyanka Chopra Jo...
2,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant,Drama,Locked Down,Anne Hathaway Chiwetel Ejiofor Stephen Merchan...
3,Simon Stone,Carey Mulligan,Ralph Fiennes,Lily James,Drama,The Dig,Carey Mulligan Ralph Fiennes Lily James Simon ...
4,Mikael Håfström,Anthony Mackie,Damson Idris,Emily Beecham,Drama,Outside the Wire,Anthony Mackie Damson Idris Emily Beecham Mika...
...,...,...,...,...,...,...,...
239,Jon Watts,Tom Holland,Zendaya,Jacob Batalon,Drama,Spider-Man: No Way Home,Tom Holland Zendaya Jacob Batalon Jon Watts Drama
240,Lana Wachowski,Keanu Reeves,Carrie-Anne Moss,Yahya Abdul-Mateen II,NaN,Untitled fourth Matrix film,NaN
241,Garth Jennings,Matthew McConaughey,Reese Witherspoon,Scarlett Johansson,Drama,Sing 2,Matthew McConaughey Reese Witherspoon Scarlett...
242,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,Drama,The King's Man,Ralph Fiennes Gemma Arterton Rhys Ifans Matthe...


In [28]:
new_df21.isna().sum()

director_name     0
actor_1_name      0
actor_2_name      2
actor_3_name     13
genres            1
movie_title       0
comb             14
dtype: int64

In [29]:
new_df21 = new_df21.dropna(how='any')

In [30]:
new_df21.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [31]:
new_df21['movie_title'] = new_df21['movie_title'].str.lower()

<ipython-input-31-6e765124d575>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df21['movie_title'] = new_df21['movie_title'].str.lower()


In [32]:
new_df21

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale,Drama,shadow in the cloud,Chloë Grace Moretz Taylor John Smith Beulah Ko...
1,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas,Drama,the white tiger,Adarsh Gourav Rajkummar Rao Priyanka Chopra Jo...
2,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant,Drama,locked down,Anne Hathaway Chiwetel Ejiofor Stephen Merchan...
3,Simon Stone,Carey Mulligan,Ralph Fiennes,Lily James,Drama,the dig,Carey Mulligan Ralph Fiennes Lily James Simon ...
4,Mikael Håfström,Anthony Mackie,Damson Idris,Emily Beecham,Drama,outside the wire,Anthony Mackie Damson Idris Emily Beecham Mika...
...,...,...,...,...,...,...,...
238,Tarik Saleh,Chris Pine,Ben Foster,Gillian Jacobs,Drama,violence of action,Chris Pine Ben Foster Gillian Jacobs Tarik Sal...
239,Jon Watts,Tom Holland,Zendaya,Jacob Batalon,Drama,spider-man: no way home,Tom Holland Zendaya Jacob Batalon Jon Watts Drama
241,Garth Jennings,Matthew McConaughey,Reese Witherspoon,Scarlett Johansson,Drama,sing 2,Matthew McConaughey Reese Witherspoon Scarlett...
242,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,Drama,the king's man,Ralph Fiennes Gemma Arterton Rhys Ifans Matthe...


In [33]:
old_df = pd.read_csv('main_data.csv')

In [34]:
old_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
5603,Pete Docter,Jamie Foxx,Tina Fey,Graham Norton,Drama,soul,Jamie Foxx Tina Fey Graham Norton Pete Docter ...
5604,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,Drama,we can be heroes,Priyanka Chopra Jonas Pedro Pascal YaYa Gossel...
5605,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,Drama,one night in miami...,Kingsley Ben-Adir Eli Goree Aldis Hodge Regina...
5606,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,Drama,promising young woman,Carey Mulligan Bo Burnham Alison Brie Emerald ...


In [35]:
final_df = old_df.append(new_df21,ignore_index=True)

In [36]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
5833,Tarik Saleh,Chris Pine,Ben Foster,Gillian Jacobs,Drama,violence of action,Chris Pine Ben Foster Gillian Jacobs Tarik Sal...
5834,Jon Watts,Tom Holland,Zendaya,Jacob Batalon,Drama,spider-man: no way home,Tom Holland Zendaya Jacob Batalon Jon Watts Drama
5835,Garth Jennings,Matthew McConaughey,Reese Witherspoon,Scarlett Johansson,Drama,sing 2,Matthew McConaughey Reese Witherspoon Scarlett...
5836,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,Drama,the king's man,Ralph Fiennes Gemma Arterton Rhys Ifans Matthe...


In [37]:
final_df.to_csv("Ending_data.csv")